In [1]:
from __future__ import division, print_function
import math
import numpy as np
import numpy.random
import scipy as sp
import scipy.stats
from scipy.optimize import minimize_scalar

import itertools

from ballot_comparison import findNmin_ballot_comparison_rates
from hypergeometric import trihypergeometric_optim, diluted_margin_trihypergeometric

# Example of a hybrid audit

There are two strata. One contains every CVR county and the other contains every non-CVR county.
There were 110,000 ballots cast in the election, 100,000 in the CVR stratum and 10,000 in the non-CVR stratum.
In the CVR stratum, there were 45,500 votes reported for A, 49,500 votes for candidate B, and 5,000 invalid ballots.
In the no-CVR stratum, there were 7,500 votes reported for A, 1,500 votes for B, and 1000 invalid ballots.
A won overall, with 53,000 votes to B's 51,000, but not in the CVR stratum.

The reported vote margin between A and B is 2000 votes, a "diluted margin" of $2000/110000 = 1.8\%$.

For any $\lambda$, the reported outcome of the election is correct if the overstatement of the margin in the CVR stratum is less than $2000\lambda$ votes and if the overstatement of the margin in the non-CVR stratum is less than $2000(1-\lambda)$ votes. 
For this example, we set $\lambda = 0.9$, roughly reflecting the relative sizes of the two strata.

We want to limit the risk of certifying an incorrect outcome to at most $\alpha=10\%$. 
We allocate risk unequally between the two strata: $\alpha_1 = 3\%$ in the CVR stratum and $\alpha_2 = 7\%$ in the non-CVR stratum; this gives an overall risk limit of $1-(1-.03)(1-.07) < 9.8\%$.

We test the following pair of null hypotheses, using independent samples from the two strata:

* the overstatment in the CVR stratum is less than $2000\lambda$. We test at significamnce level
(risk limit) $\alpha_1$ using a ballot-level comparison audit

* the overstatment in the no-CVR stratunm is less than $2000(1-\lambda)$. We test this at significance level (risk limit) $\alpha_2$ using a ballot-polling audit

If either null is not rejected, we hand count the corresponding stratum completely, adjust the null
in the other stratum to reflect the known tally in the other stratum, and then determine whether there needs to be
more auditing in the stratum that was not fully hand counted.



In [2]:
lambda1 = 0.1
lambda2 = 1-lambda1
alpha1 = 0.03
alpha2 = 0.07
margin = 2000
N1 = 100000
N2 = 10000

# CVR stratum

We compute the sample size needed to confirm the election outcome, for a number of assumed rates of error in the population of ballots.

We take the chosen $\lambda$ from above and plug it in as the parameter `null_lambda` in the function below.

We set $\gamma = 1.03905$ as in "A Gentle Introduction to Risk-limiting Audits."

In [3]:
# Assuming that the audit will find no errors
findNmin_ballot_comparison_rates(alpha=alpha1, gamma=1.03905, 
                                r1=0, s1=0, r2=0, s2=0,
                                reported_margin=margin, N=N1, null_lambda=lambda1)

35.0

In [4]:
# Assuming that the audit will find 1-vote overstatements at rate 0.1%
findNmin_ballot_comparison_rates(alpha=alpha1, gamma=1.03905, 
                                r1=0.001, s1=0, r2=0, s2=0,
                                reported_margin=margin, N=N1, null_lambda=lambda1)

35.0

# Non-CVR stratum

Below, we compute the sample size $n$ needed to confirm the election outcome.

Define
$$
    c = \text{reported margin in the stratum } - \lambda_2 \text{overall reported margin}.
$$

The reported margin in the stratum could be large or small, but it is known. 
Below, we will vary it just to see the effect.

$c$ defines the null hypothesis. We test the null that the actual margin in the stratum is less than or equal to $c$: $A_{w, 2} - A_{\ell, 2} \leq c$. Here, $A_{w, 2}$ is an unknown nuisance parameter.

In practice, we will maximize the $p$-value over all possible pairs $(A_{w,2}, A_{\ell, 2})$ in the null.

In [5]:
w = 2
l = 0
n = 4
N_w = 3
N_l = 1
N = 6
N_u = N-N_w-N_l

# answer should be (3C2*1C0*2C2 + 3C3*1C0*2C1 + 3C3*1C1*2C0)/6C4 = (3+2+1)/(6*5/2) = 2/5 = 0.4

In [6]:
diluted_margin_trihypergeometric(w, l, n, N_w, N_l, N)

0.4

In [ ]:
p = range(5)
print([r for r in p])
p = list(itertools.filterfalse(lambda y: False, p))
print([r for r in p])
print([r for r in p])

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


In [ ]:
# Assuming that the stratum reported margin is accurate

# We don't know N_w, N_\ell so maximize the p-value over all possibilities.

np.random.seed(292018)
pop = np.array([0]*1500 + [1]*7500 + [np.nan]*1000)
c = (7500-1500) - lambda2*margin
print("c= ", c)
for n in range(100, 10000, 50):
    sample = np.random.choice(pop, n)
    pval = trihypergeometric_optim(sample, popsize=N2, null_margin=c)
    print("n=", n, ", pvalue=", pval)
    if pval < 1e-3:
        break

c=  4200.0
n= 100 , pvalue= 0.118686007412
n= 150 , pvalue= 0.00141546565417


In [ ]:
# Assuming that the stratum reported margin is accurate

# Assume that we know there are 1000 invalid ballots, compute the p-value for the
# N_w and N_\ell implied by the null.

np.random.seed(292018)
pop = np.array([0]*1500 + [1]*7500 + [np.nan]*1000)
c = (7500-1500) - lambda2*margin
Nw = int(9000 - c / 2)
print("c= ", c)
for n in range(100, 10000, 50):
    pval = diluted_margin_hypergeometric(int(0.75*n/0.9), int(0.15*n/0.9), Nw, Nw-c)
    print("n=", n, ", pvalue=", pval)
    if pval < 1e-3:
        break